In [1]:
from pathlib import Path
import sqlalchemy
import pandas as pd
from utils import * 

In [2]:
queries_dir = Path('queries/')
enum_query = read_sql(queries_dir / "1.enum_query.sql")
table_query = read_sql(queries_dir / "2.table_query.sql")

datasets_dir = Path("../datasets/")
haccp_dataset_path = datasets_dir / "haccp_dataset_200525221016.csv"

In [3]:
conn = connect()

[2020-06-11 18:51:38,137][INFO|utils.py:24] >> SERVER COMNNECTED


In [4]:
initializ_db(conn)

[2020-06-11 18:51:38,335][INFO|utils.py:41] >> COMPLETE QUERY : drop table foodinfo
[2020-06-11 18:51:38,343][INFO|utils.py:41] >> COMPLETE QUERY : drop table prdkinds
[2020-06-11 18:51:38,351][INFO|utils.py:41] >> COMPLETE QUERY : drop table rawmtrls
[2020-06-11 18:51:38,361][INFO|utils.py:41] >> COMPLETE QUERY : drop table allergyprdkinds
[2020-06-11 18:51:38,371][INFO|utils.py:41] >> COMPLETE QUERY : drop table vegrestrictprdkind
[2020-06-11 18:51:38,380][INFO|utils.py:41] >> COMPLETE QUERY : drop table allergyrawmtrls
[2020-06-11 18:51:38,388][INFO|utils.py:41] >> COMPLETE QUERY : drop table vegrestrictrawmtrls
[2020-06-11 18:51:38,398][INFO|utils.py:41] >> COMPLETE QUERY : drop table usertable
[2020-06-11 18:51:38,408][INFO|utils.py:41] >> COMPLETE QUERY : drop table dietmanage
[2020-06-11 18:51:38,412][INFO|utils.py:41] >> COMPLETE QUERY : drop type allergy
[2020-06-11 18:51:38,414][INFO|utils.py:41] >> COMPLETE QUERY : drop type veg
[2020-06-11 18:51:38,416][INFO|utils.py:41] >>

In [5]:
execute_queries(conn, enum_query)
execute_queries(conn, table_query)

[2020-06-11 18:51:38,425][INFO|utils.py:41] >> COMPLETE QUERY : CREATE TYPE allergy AS ENUM ('난류', '우유', '메밀', '땅콩', '대두', '쇠고기',  '밀', '고등어', '게', '새우', '돼지고기', '복숭아', '오징어', '토마토', '아황산류', '호두', '잣', '키위', '닭고기', '조개류', '참깨')
[2020-06-11 18:51:38,427][INFO|utils.py:41] >> COMPLETE QUERY : CREATE TYPE veg AS ENUM ('vegan', 'lactoVeg', 'ovoVeg', 'lactoOvoVeg', 'pescoVeg', 'polloVeg', 'flex')
[2020-06-11 18:51:38,429][INFO|utils.py:41] >> COMPLETE QUERY : CREATE TYPE gen AS ENUM ('f', 'm')
[2020-06-11 18:51:38,443][INFO|utils.py:41] >> COMPLETE QUERY : CREATE TABLE FoodInfo (  prdlstReportNo TEXT,  prdlstName TEXT,  rawMtrl TEXT,  allergy TEXT,  nutrient TEXT,  barcode TEXT,  prdKind TEXT,  manufacture TEXT,  seller TEXT,  capacity TEXT,  imgurl1 TEXT,  imgurl2 TEXT,  primary key(prdlstReportNo) )
[2020-06-11 18:51:38,455][INFO|utils.py:41] >> COMPLETE QUERY : CREATE TABLE PrdKinds (  prdKind TEXT,  superPrdKind TEXT,  primary key(prdKind) )
[2020-06-11 18:51:38,466][INFO|utils.py:41] >

In [6]:
haccp_dataset = pd.read_csv(haccp_dataset_path)
haccp_dataset = haccp_dataset.drop(['Unnamed: 0', 'rnum', 'productgb', 'prdkindstate'], axis=1)

In [8]:
for data in haccp_dataset.values[:30]:
    reportno, prdnm, rawmtrl, allergy, nutrient, barcode, prdkind, manu, seller, capa, img1, img2 = data
    val = "'{reportno}', '{prdnm}', '{rawmtrl}', '{allergy}', '{nutrient}','{barcode}',\
    '{prdkind}', '{manu}', '{seller}', '{capa}', '{img1}', '{img2}'".format(reportno = reportno, prdnm = prdnm, rawmtrl = rawmtrl,
                                                                 allergy = allergy, nutrient = nutrient, barcode = barcode,
                                                                 prdkind = prdkind, manu=manu, seller=seller, capa=capa, img1=img1, img2=img2)
    val = val.replace('(', ' ').replace(')', ' ').replace('%', '%%')
    insert_query = "INSERT INTO foodinfo VALUES({})".format(val)
    execute_query(conn, insert_query)

 1일 영양성분 기준치에 대한 비율 %% 은 2,000 kcal 기준이므로 개인의 필요 열량에 따라 다를 수 있습니다.','8801024949960',    '두부', '대상㈜구미공장_경상북도 구미시 산동면 첨단기업5로 48-38', '대상㈜횡성공장_강원도 횡성군 횡성읍 청용길 15', '300g', 'http://fresh.haccp.or.kr/prdimg/2017/201705484642/201705484642-1.jpg', 'http://fresh.haccp.or.kr/prdimg/2017/201705484642/201705484642-2.jpg')
 1일 영양성분 기준치에 대한 비율 %% 은 2,000kcal 기준이므로 개인의 필요 열량에 따라 다를 수 있습니다.','8801024051519',    '두부', '대상㈜ 구미공장_경상북도 구미시 산동면 첨단기업5로 48-38', '대상㈜ 횡성공장_강원도 횡성군 횡성읍 청용길 15', '400g 320kcal ', 'http://fresh.haccp.or.kr/prdimg/2017/201705484641/201705484641-1.jpg', 'http://fresh.haccp.or.kr/prdimg/2017/201705484641/201705484641-2.jpg')
[2020-06-11 18:51:45,191][INFO|utils.py:41] >> COMPLETE QUERY : INSERT INTO foodinfo VALUES('201705037815', '목포미항 재래김', '재래김65%%,옥배유 수입산30.4%% ,참기름 인도산2%% ,천일염 국내산2%% ,생강0.3%%,감초뿌리 0.3%% ', '알수없음', '알수없음','알수없음',    '조미김', '유한회사 청해에스앤에프_전남 목포시 전라남도 목포시 대양산단로125번길 139  대양동 ', '_', '20g 100kcal ', 'http://fresh.haccp.or.kr/prdimg/2017/201705037815/201705037815-1.j